In [0]:
pip install firebase_admin

     |████████████████████████████████| 92kB 3.1MB/s 
     |████████████████████████████████| 337kB 8.9MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
ERROR: google-cloud-storage 1.25.0 has requirement google-auth<2.0dev,>=1.9.0, but you'll have google-auth 1.4.2 which is incompatible.
ERROR: google-cloud-storage 1.25.0 has requirement google-cloud-core<2.0dev,>=1.2.0, but you'll have google-cloud-core 1.0.3 which is incompatible.
ERROR: google-cloud-bigquery 1.21.0 has requirement google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing installation: google-cloud-storage 1.16.2
    Uninstalling google-cloud-storage-1.16.2:
      Successfully uninstalled google-cloud-storage-1.16.2


In [0]:
from keras.layers import Dense, LSTM, Input, Dropout, Bidirectional
from keras.models import Model, load_model
import numpy as np
import random as rand
from math import log10, log
import datetime
import firebase_admin
from firebase_admin import credentials, storage
from urllib.request import urlretrieve
from flask import Flask, request
import json
import threading
import requests

Using TensorFlow backend.


In [0]:
def retrain(model, data, epochs):
  x_train = data[0]
  y_train = data[1]
  model.fit(np.array(x_train), np.array(y_train), epochs=epochs)
  return model

In [0]:
def predict(model, previous_grades, required_grades, timesteps):
  variation = 0.1
  generations = 3
  best = [0 if required_grades[i] == 0 else required_grades[i] - previous_grades[i] for i in range(15)]
  best.append(1)
  variations = [0 if i == 0 else variation for i in previous_grades]
  variations.append(variation)
  random_outputs, no_tries = [], 30
  for output in range(no_tries):
    temp = []
    for i in range(timesteps):
      index = np.argmax(np.random.normal(best, variations))
      temp.append(np.array([0 if i != index else 1 for i in range(16)]))
    random_outputs.append(np.asarray(temp))
  errors = []
  for output in random_outputs:
    y_true, y_pred = required_grades, model.predict(np.array([output]))[0]
    error = sum([((y_true[i] - y_pred[i] + abs(y_true[i] - y_pred[i])) ** 2) / 4 for i in range(len(y_true))])
    errors.append(error)
  best = random_outputs[np.argmin(errors)]
  for i in range(generations - 1):
    random_outputs, no_tries = [best], 30
    for output in range(no_tries):
      temp = []
      for i in range(timesteps):
        index = np.argmax(np.random.normal(best[i], variations))
        temp.append(np.array([0 if i != index else 1 for i in range(16)]))
      random_outputs.append(np.asarray(temp))
    errors = []
    for output in random_outputs:
      y_true, y_pred = required_grades, model.predict(np.array([output]))[0]
      error = sum([((y_true[i] - y_pred[i] + abs(y_true[i] - y_pred[i])) ** 2) / 4 for i in range(len(y_true))])
      errors.append(error)
    best = random_outputs[np.argmin(errors)]
  return best

In [0]:
def start(subjects):
  attainments = [5,6,7,8]
  models = []
  for attainment in attainments:
    x_train, y_train, x_test, y_test = simulate_users(attainment, subjects)
    input_shape = None, x_train.shape[2]
    output_shape = y_train.shape[1] 
    model = build_model(input_shape, output_shape)
    model.fit(x_train, y_train, epochs=250, batch_size=2800, validation_data=(x_test, y_test))
    models.append(model)
  return models

In [0]:
def simulate_users(attainment, subjects):
  variation = 0
  datapoints, periods = 3000, 10
  x, subject_idxs = generate_input_data(subjects, periods, datapoints, attainment)
  y = generate_output_data(x, attainment, variation, subject_idxs)
  y = format_output_data(y)
  x_test, x_train = x[:200], x[200:]
  y_test, y_train = y[:200], y[200:]
  return x_train, y_train, x_test, y_test

In [0]:
def build_model(input_shape, output_shape):
  input_subject = Input(shape=(input_shape))
  layer_1 = Bidirectional(LSTM(128, recurrent_dropout=0.7))(input_subject)
  output = Dense(output_shape, activation='sigmoid')(layer_1)
  model = Model(input_subject, output)
  model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
def generate_input_data(subjects, periods, datapoints, attainment):
  x, list_of_subject_idxs = [], []
  for i in range(datapoints):
    subject_idxs = rand.sample(range(15), subjects)
    subject_idxs_with_rests = [i for i in subject_idxs]
    subject_idxs_with_rests.append(15)
    list_of_subject_idxs.append(subject_idxs)
    datapoint = []
    for i in range(periods):
      subject_idx = rand.choice(subject_idxs_with_rests)
      subject_ohe = np.linspace(0, 0, 16)
      subject_ohe[subject_idx] = 1
      datapoint.append(subject_ohe)
    datapoint = np.asarray(datapoint)
    x.append(datapoint)
  x = np.asarray(x)
  return x, list_of_subject_idxs

In [0]:
def generate_output_data(x, attainment, variation, subject_idxs):
  dist_weighting = -0.02
  adj_break_weighting = 0.6
  adj_subj_weighting = -0.2
  y = []
  for j, datapoint in enumerate(x):
    length = len(datapoint)
    efficient_time_studied, grade = [0 for i in range(16)], [0 for i in range(15)]
    for i, period in enumerate(datapoint):
      distance = length - i
      rest, same = 0, 0
      if np.argmax(datapoint[i - 1]) == 15:
        rest = 1
      elif np.argmax(datapoint[i - 1]) == np.argmax(period):
        same = 1
      efficient_time_studied[np.argmax(period)] += 1 + dist_weighting * distance + adj_break_weighting * rest + adj_subj_weighting * same
    for i in subject_idxs[j]:
      grade[i] = np.random.normal((2.37089 + 0.602646 * attainment + (3.63039 - 0.330035 * attainment) * log(0.3 + 0.9 * efficient_time_studied[i])), variation)
    for i in range(len(grade)):
      if grade[i] == 0.:
        grade[i] = 5.
    y.append(grade)
  y = np.asarray(y)
  return y

In [0]:
def format_output_data(y_unformatted):
  y_formatted = y_unformatted / 9
  return y_formatted

In [0]:
def generate_timetable(inputs):
  epochs = len(inputs["training_data"]["x"]) * 5 ## this multiplier can be optimised
  if inputs["training_data"]["x"] != []:
    model = retrain(load_model("model_attainment_" + inputs["attainment"] + ".h5"), (inputs["training_data"]["x"], inputs["training_data"]["y"]), epochs)
  else:
    model = load_model("model_attainment_" + inputs["attainment"] + ".h5")
  output = predict(model, inputs["prediction_data"]["past_grades"], inputs["prediction_data"]["required_grades"], inputs["prediction_data"]["hours"])
  return output

In [0]:
## To make the models (they should be in firebase so this is commented out)
'''
models = start(8)
for i in range(4):
  filename = "model_attainment_" + str(i + 5) + ".h5"
  models[i].save(filename)
'''

## firebase credentials
credentials_for_json = {
  "type": "service_account",
  "project_id": "maximise-uk",
  "private_key_id": "REMOVED",
  "private_key": "REMOVED",
  "client_email": "REMOVED",
  "client_id": "REMOVED",
  "auth_uri": "REMOVED",
  "token_uri": "REMOVED",
  "auth_provider_x509_cert_url": "REMOVED",
  "client_x509_cert_url": "REMOVED"
}

with open("credentials.json", "w") as write_file:
    json.dump(credentials_for_json, write_file)

## load models from firebase on startup
cred = credentials.Certificate("credentials.json")
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'maximise-uk.appspot.com',
}, name='storage')
bucket = storage.bucket(app=app)
for i in range(5, 9):
  blob = bucket.blob(f"model_attainment_{i}.h5")
  urlretrieve(blob.generate_signed_url(datetime.timedelta(seconds=300), method='GET'), f'model_attainment_{i}.h5')

## run flask program that runs the ml function
#print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))
app = Flask(__name__)
@app.route("/", methods=["POST"])
def generate_ml():
    inputs = request.get_json()
    output = generate_timetable(inputs)
    return str(output)
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80}).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production


In [0]:
## make a request
inputs = {
  "attainment": "8",
  "training_data": {
      "x": [[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]],
      "y": [[8, 5, 5, 5, 8, 9, 7, 5, 5, 5, 5, 5, 8, 5, 5]]
  },
  "prediction_data": {
    "past_grades": [6, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 6, 0, 0],
    "required_grades": [7, 0, 0, 0, 7, 7, 7, 0, 0, 0, 0, 0, 7, 0, 0],
    "hours": 20
  }
}
r = requests.post("http://172.28.0.2/", json = inputs)
print(r.text)

   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: 241.3333 - acc: 1.0000
Epoch 2/5
1/1 [==============================] - 0s 20ms/step - loss: 241.2560 - acc: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 18ms/step - loss: 241.2668 - acc: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 19ms/step - loss: 241.4068 - acc: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 18ms/step - loss: 241.3922 - acc: 1.0000


172.28.0.2 - - [20/Jan/2020 12:49:19] "POST / HTTP/1.1" 200 -


[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
